In [7]:
import pandas as pd

# usecols will specify columns we want to add from ths csv to our dataframe
imdb = pd.read_csv("IMDB_movies.csv", usecols=['title','year','date_published','genre','duration','country','language','director','actors','avg_vote','votes','budget','usa_gross_income','worldwide_gross_income','metascore','reviews_from_users','reviews_from_critics'])
imdb

,title,year,date_published,genre,duration,country,language,director,actors,avg_vote,votes,budget,usa_gross_income,worldwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,"Blanche Bayliss, William Courtenay, Chauncey D...",5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",6.1,589,"$2,250",NaN,NaN,NaN,7.0,7.0
2,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",5.2,446,"$45,000",NaN,NaN,NaN,25.0,3.0
4,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan","Salvatore Papa, Arturo Pirovano, Giuseppe de L...",7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85850,Le lion,2020,2020-01-29,Comedy,95,"France, Belgium",French,Ludovic Colbeau-Justin,"Dany Boon, Philippe Katerine, Anne Serra, Samu...",5.3,398,NaN,NaN,"$3,507,171",NaN,NaN,4.0
85851,De Beentjes van Sint-Hildegard,2020,2020-02-13,"Comedy, Drama",103,Netherlands,"German, Dutch",Johan Nijenhuis,"Herman Finkers, Johanna ter Steege, Leonie ter...",7.7,724,NaN,NaN,"$7,299,062",NaN,6.0,4.0
85852,Padmavyuhathile Abhimanyu,2019,2019-03-08,Drama,130,India,Malayalam,Vineesh Aaradya,"Anoop Chandran, Indrans, Sona Nair, Simon Brit...",7.9,265,NaN,NaN,NaN,NaN,NaN,NaN
85853,Sokagin Çocuklari,2019,2019-03-15,"Drama, Family",98,Turkey,Turkish,Ahmet Faik Akinci,"Ahmet Faik Akinci, Belma Mamati, Metin Keçeci,...",6.4,194,NaN,NaN,"$2,833",NaN,NaN,NaN
